In [10]:
import yfinance as yf
import pandas as pd
import configparser
import sqlalchemy

In [11]:
msft = yf.Ticker("MSFT")

In [ ]:
msft.info

In [ ]:
msft.info.keys()

In [14]:
print(msft.info["symbol"])
print(msft.info["longName"])
print(msft.info["country"])

MSFT
Microsoft Corporation
United States


In [15]:
config = configparser.ConfigParser()
config.read("config.ini")
user = config["local_postgres"]["DB_USER"]
pw = config["local_postgres"]["DB_PW"]
address = config["local_postgres"]["DB_ADDRESS"]
port = config["local_postgres"]["DB_PORT"]
name = config["local_postgres"]["DB_NAME"]

con_str = f"postgresql://{user}:{pw}@{address}:{port}/{name}"
con = sqlalchemy.create_engine(con_str)

In [16]:
query = """insert into finance.symbols values 
    ('MSFT', 'Microsoft', 'United States');"""
con.execute(query)

In [ ]:
symbol = msft.info["symbol"]
name = msft.info["longName"]
country = msft.info["country"]
query = f"""insert into finance.symbols values 
    ('{symbol}', '{name}', '{country}');"""
con.execute(query)

In [18]:
def insert_symbol(con, ticker):
    # Daten aus Ticker auslesen
    symbol = ticker.info["symbol"]
    name = ticker.info["longName"]
    country = ticker.info["country"]
    # Daten in Datenbank schreiben
    query = f"""insert into finance.symbols values 
        ('{symbol}', '{name}', '{country}');"""    
    con.execute(query)

t = yf.Ticker("AAPL")
insert_symbol(con, t)

In [ ]:
insert_symbol(con, t)

In [20]:
symbol = 'MSFT'
query = f"""select count(1) from finance.symbols 
                where symbol = '{symbol}';"""
df = pd.read_sql(query,con)
df.iloc[0,0] == 1

True

In [21]:
def insert_symbol(con, ticker):
    # Daten aus Ticker auslesen
    symbol = ticker.info["symbol"]
    name = ticker.info["longName"]
    country = ticker.info["country"]
    # Prüfen, ob Symbol schon vorhanden
    query = f"""select count(1) from finance.symbols 
                where symbol = '{symbol}';"""
    df = pd.read_sql(query,con)
    vorhanden = df.iloc[0,0] == 1
    # Daten in Datenbank schreiben
    if not vorhanden:
        query = f"""insert into finance.symbols values 
            ('{symbol}', '{name}', '{country}');"""    
        con.execute(query)
    else:
        print("Eintrag existiert schon")

In [22]:
t = yf.Ticker("AAPL")
insert_symbol(con, t)

Eintrag existiert schon
